# Data Tabular

In [1]:
# Import library yang dibutuhkan
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, LeakyReLU
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

In [2]:
#Memuat Dataset
data = pd.read_csv('/content/drive/MyDrive/Machine Learning/Modul 1/obesity_data.csv')
data

,Age,Gender,Height,Weight,BMI,PhysicalActivityLevel,ObesityCategory
0,56,Male,173.575262,71.982051,23.891783,4,Normal weight
1,69,Male,164.127306,89.959256,33.395209,2,Obese
2,46,Female,168.072202,72.930629,25.817737,4,Overweight
3,32,Male,168.459633,84.886912,29.912247,3,Overweight
4,60,Male,183.568568,69.038945,20.487903,3,Normal weight
...,...,...,...,...,...,...,...
995,18,Male,155.588674,64.103182,26.480345,4,Overweight
996,35,Female,165.076490,97.639771,35.830783,1,Obese
997,49,Female,156.570956,78.804284,32.146036,1,Obese
998,64,Male,164.192222,57.978115,21.505965,4,Normal weight


In [3]:
# Encoding label kategorikal
label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])  # 0 untuk Female, 1 untuk Male
data['ObesityCategory'] = label_encoder.fit_transform(data['ObesityCategory'])  # Mengubah kelas menjadi angka


In [4]:
data

,Age,Gender,Height,Weight,BMI,PhysicalActivityLevel,ObesityCategory
0,56,1,173.575262,71.982051,23.891783,4,0
1,69,1,164.127306,89.959256,33.395209,2,1
2,46,0,168.072202,72.930629,25.817737,4,2
3,32,1,168.459633,84.886912,29.912247,3,2
4,60,1,183.568568,69.038945,20.487903,3,0
...,...,...,...,...,...,...,...
995,18,1,155.588674,64.103182,26.480345,4,2
996,35,0,165.076490,97.639771,35.830783,1,1
997,49,0,156.570956,78.804284,32.146036,1,1
998,64,1,164.192222,57.978115,21.505965,4,0


In [5]:
# Pilih kolom yang akan distandarisasi
columns_to_scale = ['Height', 'Weight', 'BMI']

# Inisialisasi Min-Max Scaler
scaler = StandardScaler()
X = scaler.fit_transform(data.drop('ObesityCategory', axis=1))
y = data['ObesityCategory'].values

# Terapkan Min-Max Scaling hanya pada kolom yang dipilih
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

In [6]:
data

,Age,Gender,Height,Weight,BMI,PhysicalActivityLevel,ObesityCategory
0,56,1,0.341864,0.050076,-0.160970,4,0
1,69,1,-0.574985,1.209739,1.374115,2,1
2,46,0,-0.192164,0.111266,0.150129,4,2
3,32,1,-0.154567,0.882535,0.811514,3,2
4,60,1,1.311635,-0.139776,-0.710797,3,0
...,...,...,...,...,...,...,...
995,18,1,-1.403591,-0.458170,0.257160,4,2
996,35,0,-0.482874,1.705189,1.767533,1,1
997,49,0,-1.308268,0.490161,1.172337,1,1
998,64,1,-0.568685,-0.853282,-0.546350,4,0


In [7]:
# Pisahkan fitur dan label
X = data.drop('ObesityCategory', axis=1)
y = data['ObesityCategory']

In [8]:
# Split data menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [55]:
# Pastikan X_train adalah numpy array
X_train_array = X_train.values if isinstance(X_train, pd.DataFrame) else X_train

class GAN():
    def __init__(self, data):
        self.data = data

        self.generator = self._build_generator()
        self.discriminator = self._build_discriminator()
        self.cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
        self.generator_optimizer = tf.keras.optimizers.Adam(1e-4)
        # self.discriminator.trainable = False
        self.discriminator_optimiezer = tf.keras.optimizers.Adam(1e-4)

        noise = tf.keras.layers.Input(shape=(self.data,))
        generated_data = self.generator(noise)
        validity = self.discriminator(generated_data)

        self.combined = tf.keras.models.Model(noise, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer='adam')

    def _build_generator(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(32, activation='relu', input_dim=self.data),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(self.data, activation='tanh')
        ])
        return model

    def _build_discriminator(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_dim=self.data),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        model.compile(loss='binary_crossentropy', optimizer='adam')
        return model

    @tf.function
    def train_step(self):
        idx = np.random.randint(0, X_train_array.shape[0])
        real_data = X_train_array[idx]  # Use actual training data

        noise = tf.random.normal([self.data])
        generated_data = self.generator(noise, training=True)

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            real_output = self.discriminator(real_data, training=True)
            fake_output = self.discriminator(generated_data, training=True)

            gen_loss = self.cross_entropy(tf.ones_like(fake_output), fake_output)
            disc_loss = self.cross_entropy(tf.ones_like(real_output), real_output) + \
                        self.cross_entropy(tf.zeros_like(fake_output), fake_output)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))

        return gen_loss, disc_loss

    # Training Loop
    def train(self, epochs):
        for epoch in range(epochs):
          gen_loss, disc_loss = self.train_step()
          print(f"{epoch} [D loss: {disc_loss:.4f}] [G loss: {gen_loss:.4f}]")

In [56]:
# Instantiate and train GAN
model = GAN(X_train_array.shape[1])
model.train(epochs=100)

noise = np.random.normal(0, 1, (X_test.shape[0], GAN.data_dim))
generated_data = GAN.generator.predict(noise)

ValueError: in user code:

    File "<ipython-input-53-fd3bbd0a99c1>", line 45, in train_step  *
        generated_data = self.generator(noise, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/models/functional.py", line 244, in _adjust_input_rank
        raise ValueError(

    ValueError: Exception encountered when calling Sequential.call().
    
    [1mInvalid input shape for input Tensor("random_normal:0", shape=(6,), dtype=float32). Expected shape (None, 6), but input has incompatible shape (6,)[0m
    
    Arguments received by Sequential.call():
      • inputs=tf.Tensor(shape=(6,), dtype=float32)
      • training=True
      • mask=None


In [ ]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model on augmented data
history = model.fit(X_train_augmented, y_train_augmented, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)


In [ ]:
# Evaluate the model
y_pred = np.argmax(model.predict(X_test), axis=1)
print("Classification Report:\n", classification_report(y_test, y_pred))